In [ ]:
!mkdir -P ../downloads/zips/
!mkdir -P ../downloads/raw_audio/

In [ ]:
import pandas as pd

df = pd.read_csv("../data/openbible_swahili_urls.tsv", sep="\t", names=["book", "url"])

In [ ]:
for url in df["url"]:
    !wget {url} -P ../downloads/zips/

In [2]:
from glob import glob
from tqdm.auto import tqdm

zips = sorted(glob("../downloads/zips/*"))
for z in tqdm(zips):
    o = z.split("/")[-1]
    !mkdir ../downloads/raw_audio/{o}
    !unzip -q {z} -d ../downloads/raw_audio/{o}

/root/miniconda3/envs/vits2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 66/66 [00:00<00:00, 788672.55it/s]


In [11]:
from pydub import AudioSegment
from pathlib import Path
from tqdm.contrib.concurrent import process_map

def convert_mp3_to_wav(mp3_path):
    chapter = mp3_path.split("/")[-1].strip(".mp3")
    book = chapter.split("_")[0]

    output_path_16 = f"../downloads/wavs_16/{book}/{chapter}.wav"
    Path(output_path_16).parent.mkdir(parents=True, exist_ok=True)
    output_path_44 = f"../downloads/wavs_44/{book}/{chapter}.wav"
    Path(output_path_44).parent.mkdir(parents=True, exist_ok=True)

    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(output_path_16, format="wav", parameters=["-ar", "16000", "-ac", "1"])
    audio.export(output_path_44, format="wav", parameters=["-ar", "44100", "-ac", "1"])

raw_audios = sorted(glob("../downloads/raw_audio/*/*.mp3"))
_ = process_map(convert_mp3_to_wav, raw_audios)

/tmp/ipykernel_9846/3452034609.py:19: TqdmWarning: Iterable length 1189 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  _ = process_map(convert_mp3_to_wav, raw_audios)
  0%|          | 0/1189 [00:00<?, ?it/s]

100%|██████████| 1189/1189 [02:50<00:00,  6.97it/s]


In [1]:
from glob import glob

usxs = sorted(glob("../downloads/release/USX_1/*.usx"))
usxs[:3]

['../downloads/release/USX_1/1CH.usx',
 '../downloads/release/USX_1/1CO.usx',
 '../downloads/release/USX_1/1JN.usx']

In [73]:
import re

def parse_usx(usx_path):
    # with open(usx_path) as f:
    #     lines = re.sub(r"\s+", " ", f.read())
    lines = """<verse number="6" style="v" sid="MRK 6:6"/>Naye akashangazwa sana kwa jinsi wasivyokuwa na imani.</para>
  <para style="s1" vid="MRK 6:6">Yesu Awatuma Wale Kumi Na Wawili</para>
  <para style="r" vid="MRK 6:6">(<ref loc="MAT 10:5-15">Mathayo 10:5-15</ref>; <ref loc="LUK 9:1-6">Luka 9:1-6</ref>)</para>
  <para style="p" vid="MRK 6:6">Kisha Yesu akawa anakwenda kutoka kijiji kimoja hadi kingine akifundisha. <verse eid="MRK 6:6"/>"""
    lines = re.sub(r"\s+", " ", f.read())

    verse_matches = re.findall(r"<verse.+?sid=\"(.+?)\"\/>(.*?)<verse eid=\".+?\"\/>", lines)
    if verse_matches:
        for match in verse_matches:
            vid = match[0]
            verse_text = match[1]
            for para_matches in re.findall(r"(<\/para>\s+<para.+?vid=\".+?\">)", verse_text):
                verse_text = verse_text.replace(para_matches, " ")
            for note_matches in re.findall(r"(<note.+?<\/note>)", verse_text):
                verse_text = verse_text.replace(note_matches, " ")
            for char_matches in re.findall(r"(<char.+?>(.+?)<\/char>)", verse_text):
                verse_text = verse_text.replace(char_matches[0], char_matches[1])
            for char_matches in re.findall(r"(<char.+?>(.+?)<\/char>)", verse_text):
                verse_text = verse_text.replace(char_matches[0], char_matches[1])
            for cell_matches in re.findall(r"(<cell.+?>(.+?)<\/cell>)", verse_text):
                verse_text = verse_text.replace(cell_matches[0], cell_matches[1])
            for cell_matches in re.findall(r"(<cell.+?>)", verse_text):
                verse_text = verse_text.replace(cell_matches, " ")
            for row_matches in re.findall(r"(<row.+?>)", verse_text):
                verse_text = verse_text.replace(row_matches, " ")
            # for para_matches in re.findall(r"(<para style=\".\" vid=\".+?\">)", verse_text):
            #     verse_text = verse_text.replace(para_matches, " ")
            REMOVE_TAGS = ["<optbreak/>", "</cell>", "</row>", "</table>", "<table>", "</para>"]
            for tag in REMOVE_TAGS:
                verse_text = verse_text.replace(tag, " ")
            verse_text = re.sub(r"\s+", " ", verse_text).strip()
            if "<" in verse_text:
                print(usx_path)
                print(vid, verse_text)
            # print(vid, verse_text)

In [74]:
parse_usx(None)

ValueError: I/O operation on closed file.

In [70]:
# for usx_path in usxs:
#     parse_usx(usx_path)

../downloads/release/USX_1/1CH.usx
1CH 25:31 ya ishirini na nne ikamwangukia Romamti-Ezeri, wanawe na jamaa zake, 12. <chapter eid="1CH 25"/> <chapter number="26" style="c" sid="1CH 26"/> <para style="s1" vid="1CH 25:31">Mabawabu


../downloads/release/USX_1/EZR.usx
EZR 1:10 mabakuli ya dhahabu yalikuwa 30 mabakuli ya fedha yanayofanana yalikuwa 410 vifaa vingine vilikuwa 1,000. <para style="b" vid="EZR 1:10">
../downloads/release/USX_1/EZR.usx
EZR 2:20 wazao wa Gibari 95 <para style="b" vid="EZR 2:20">
../downloads/release/USX_1/EZR.usx
EZR 2:35 wazao wa Senaa 3,630 <para style="b" vid="EZR 2:35">
../downloads/release/USX_1/EZR.usx
EZR 2:39 wazao wa Harimu 1,017 <para style="b" vid="EZR 2:39">
../downloads/release/USX_1/EZR.usx
EZR 2:40 Walawi: wazao wa Yeshua na Kadmieli (kupitia jamaa ya Hodavia) 74 <para style="b" vid="EZR 2:40">
../downloads/release/USX_1/EZR.usx
EZR 2:41 Waimbaji: wazao wa Asafu 128 <para style="b" vid="EZR 2:41">
../downloads/release/USX_1/EZR.usx
EZR 2:42 Mabawabu wa lango la Hekalu: wazao wa Shalumu, Ateri, Talmoni, Akubu, Hatita na Shobai 139 <para style="b" vid="EZR 2:42">
../downloads/release/USX_1/EZR.usx
EZR 2:58 Watumishi wa Hekalu wote na wazao wa watumishi wa Solomoni 392 <para s